# Use LLMs to query Database - *Text-to-Query*

This notebook demonstrates a simple workflow which uses **Vanna** to turn English sentences into SQL.

Large Language Models (LLMs) are getting increasingly popular. It is now possible to generate web applications, create apps and manipulate large-scale data using models like ChatGPT, Google Bard, and DALL-E.

It is now also possible to build a *Text-to-Query* system using Natural Language Processing (NLP) techniques. 

I will 

* briefly introduce the concept using **Venna** as an example
* then move back to the created DuckDB table for a small demonstration
* finally, I want to highlight the challenges and limitations of using such a technique in the data analytics field. 
                                                                            


## Why **Venna**
I select **Vanna** mainly for demonstration purposes. There are other solutions like  __[AWS + Claudev2](https://aws.amazon.com/blogs/machine-learning/build-a-robust-text-to-sql-solution-generating-complex-queries-self-correcting-and-querying-diverse-data-sources/)__ and __[LangChain](https://github.com/langchain-ai/langchain)__. The problem is they require API keys and they are not free. 

Vanna,  also requires an API key, including an OpenAI key, but it is available for free for at least one month.
For more information about it please go to __[Venna.AI](https://vanna.ai/docs/postgres-openai-vanna-vannadb/)__
Their GitHbub is __[https://github.com/vanna-ai/vanna](https://github.com/vanna-ai/vanna)__.
Make sure you obtain your API key and model name according to 
* LLM = OpenAI via Vanna.AI (Recommended)
* training data = Vanna Hosted Vector DB (Recommended)
* Database = DuckDB


![How Venna works](img/1_KZNioLcEiA0y-cYn-Astsg.webp)

Now, let's look at how we query our listingGrpMonthly table using text.                                                                                                


In [ ]:
# %pip install vanna


In [2]:
import vanna
from vanna.remote import VannaDefault

Here we create a default **Vanna** default model. 
To use the specific model you created online in previous step, you specify your api_key and model_name.

In [3]:
api_key = 'aa7f794af5d342d0a13c590204992cd0' # Your API key from https://vanna.ai/account/profile 

vanna_model_name ='yang_test' # Your model name from https://vanna.ai/account/profile 
vn = VannaDefault(model=vanna_model_name, api_key=api_key)

In case we mess up the database created in the previous step and demonstrate how you can retrieve extra data from another database. </br>
Notice that we connect to a new database called 'dbtrain.db' first. </br>
Then **Attach** the previous database, my_database3.db, and call it *duck*. </br>
We **Create Table** *ListingIZMonthly* based on *listingGrpMonthly* in *duck*. </br>

In [4]:
import duckdb as dd
# con.close()
con=dd.connect('my_database_exercise.db')
con.sql('SHOW TABLES;')
con.sql("select count(*) from listingGrpMonthly")
# con.close()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│         5396 │
└──────────────┘

In [6]:
import duckdb as dd
con.close()
con=dd.connect('dbtrain_exercise.db')
con.sql('SHOW TABLES;')
con.sql("""SELECT database_name, path, type FROM duckdb_databases;""")

con.sql("""ATTACH 'duckdb:my_database_exercise.db' AS duck;""")
con.sql("""SELECT database_name, path, type FROM duckdb_databases;""")

con.sql("""CREATE TABLE listingIZMonthly as FROM duck.listingGrpMonthly;""")
con.sql("""DETACH duck""")

con.sql("""show tables;""")
con.sql("""select * from listingIZMonthly""")


┌──────────┬───────────┬───────┬─────────────────────────────────────────┬─────────────────────┐
│ listings │   month   │ year  │            intermediateZone             │    listingRatio     │
│  int64   │  varchar  │ int64 │                 varchar                 │       double        │
├──────────┼───────────┼───────┼─────────────────────────────────────────┼─────────────────────┤
│       37 │ April     │  2021 │ Craigleith, Orchard Brae and Crewe Toll │  1.7961165048543688 │
│       14 │ April     │  2021 │ Gracemount, Southhouse and Burdiehouse  │   0.596252129471891 │
│       32 │ April     │  2021 │ Prestonfield                            │  1.9047619047619049 │
│        8 │ April     │  2021 │ Hyvots and Gilmerton                    │  0.5063291139240507 │
│        7 │ April     │  2021 │ Currie East                             │  0.5498821681068342 │
│       11 │ April     │  2021 │ South Gyle                              │   0.507380073800738 │
│       22 │ April     │  2021

Now, we can see in the new *dbtrain.db*, there is a new table called *ListingIZMonthly*. 

In [7]:
con.sql("""show tables;""")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ listingIZMonthly │
└──────────────────┘

Now, we are ready to build the model. First, we connect the dbtrain.db to our Vanna model.

In [8]:
vn.connect_to_duckdb('dbtrain_exercise.db')


True


In [9]:
con.sql("""show tables;""")

┌──────────────────┐
│       name       │
│     varchar      │
├──────────────────┤
│ listingIZMonthly │
└──────────────────┘

In [10]:
con.sql("""select * from listingIZMonthly """)

┌──────────┬───────────┬───────┬─────────────────────────────────────────┬─────────────────────┐
│ listings │   month   │ year  │            intermediateZone             │    listingRatio     │
│  int64   │  varchar  │ int64 │                 varchar                 │       double        │
├──────────┼───────────┼───────┼─────────────────────────────────────────┼─────────────────────┤
│       37 │ April     │  2021 │ Craigleith, Orchard Brae and Crewe Toll │  1.7961165048543688 │
│       14 │ April     │  2021 │ Gracemount, Southhouse and Burdiehouse  │   0.596252129471891 │
│       32 │ April     │  2021 │ Prestonfield                            │  1.9047619047619049 │
│        8 │ April     │  2021 │ Hyvots and Gilmerton                    │  0.5063291139240507 │
│        7 │ April     │  2021 │ Currie East                             │  0.5498821681068342 │
│       11 │ April     │  2021 │ South Gyle                              │   0.507380073800738 │
│       22 │ April     │  2021

After reminding us what is inside the *listingIZMonthly* table, we start to organize a key query which is used to retrieve information from the table. </br>
We want to show the listing density at a monthly basis.

In [11]:
con.sql("""
select month, year, listingRatio, intermediateZone from listingIZMonthly
where month='May' and year=2023
order by listingRatio asc
""")


┌─────────┬───────┬─────────────────────┬───────────────────────────────────────────┐
│  month  │ year  │    listingRatio     │             intermediateZone              │
│ varchar │ int64 │       double        │                  varchar                  │
├─────────┼───────┼─────────────────────┼───────────────────────────────────────────┤
│ May     │  2023 │ 0.11074197120708748 │ Fairmilehead                              │
│ May     │  2023 │   0.291970802919708 │ East Craigs South                         │
│ May     │  2023 │  0.3142183817753339 │ Currie East                               │
│ May     │  2023 │   0.326530612244898 │ Mortonhall and Anwickhill                 │
│ May     │  2023 │  0.3811944091486658 │ Barnton, Cammo and Cramond South          │
│ May     │  2023 │ 0.47343503419253025 │ East Craigs North                         │
│ May     │  2023 │  0.4908522980812137 │ Comiston and Swanston                     │
│ May     │  2023 │  0.5096262740656852 │ Longstone an

In [ ]:
# not needed as the table created in this way
# con.sql("""ALTER TABLE listingIZMonthly RENAME dm TO month;""")
# con.sql("""ALTER TABLE listingIZMonthly RENAME dy TO year;""")
# con.sql("""ALTER TABLE listingIZMonthly ADD COLUMN ratioOfListings FLOAT DEFAULT (listings/HH)*100;""")
# con.sql("""ALTER TABLE listingIZMonthly RENAME Name TO IntermediateZone;""")



In [12]:
vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

,table_catalog,table_schema,table_name,column_name,ordinal_position,column_default,is_nullable,data_type,character_maximum_length,character_octet_length,...,identity_generation,identity_start,identity_increment,identity_maximum,identity_minimum,identity_cycle,is_generated,generation_expression,is_updatable,COLUMN_COMMENT
0,dbtrain_exercise,main,listingIZMonthly,listings,1,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
1,dbtrain_exercise,main,listingIZMonthly,month,2,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
2,dbtrain_exercise,main,listingIZMonthly,year,3,None,YES,BIGINT,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
3,dbtrain_exercise,main,listingIZMonthly,intermediateZone,4,None,YES,VARCHAR,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None
4,dbtrain_exercise,main,listingIZMonthly,listingRatio,5,None,YES,DOUBLE,NaN,NaN,...,None,None,None,None,None,NaN,None,None,NaN,None


The schema of the database is very simple. We decide to use it as the generic training plan. </br>
If you have a very complicated schema, e.g. several tables with keys joining them, you may consider the part which is useful to the model. 

In [13]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)
plan

# If you like the plan, then uncomment this and run it to train
vn.train(plan=plan)

Our main training data is the *listingIZMonthly* table which shall be passed as ddl.

In [14]:
vn.train(ddl="DESCRIBE SELECT * FROM listingIZMonthly;")

Adding ddl: DESCRIBE SELECT * FROM listingIZMonthly;


'495684-ddl'

We then create a text vs sql pair as the training set. 
This part is critical for the LLM to match the embeddings between text and sql sentences. 

In [15]:
# here is an example of training on SQL statements
# In the listing ratio query of listingIZMonthly table.
vn.train(
question="listingRatio in intermediateZone every month"
,sql="""
    select month, year, listingRatio, intermediateZone from listingIZMonthly
    order by listingRatio asc
;
""")

'4ff618b2b0448cc1b570f35ce6af1e55-sql'

We also can add a document as another form of training set. 

In [16]:
vn.train(documentation="The ratio of listings in intermediate zones (IZs)")

Adding documentation....


'3124640-doc'

After these simple training steps, we are ready to run our model.</br>
We want to ask for some information about the listing ratio in different time frames and intermediate zones.</br
You will see some details about the transformation of query and data.</br>
An output is an SQL sentence the model generated based on the prompt.

In [28]:
vn.ask("Which intermediateZone had the highest listingRatio in 2020? what is the month?", visualize=False,allow_llm_to_see_data=True)


SQL Prompt: [{'role': 'system', 'content': "You are a DuckDB SQL expert. Please help to generate a SQL query to answer the question. Your response should ONLY be based on the given context and follow the response guidelines and format instructions. \n===Tables \nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingIZMonthly;\n\nDESCRIBE SELECT * FROM listingGrpMonthly;\n\nDESCRIBE SELECT * FROM listingGrpMonthly;\n\n\n===Additional Context \n\nThe ratio of listings in intermediate zones (IZs) over time\n\nThe ratio of listings in intermediate zones (IZs) over time\n\nThe ratio of listings in intermediate zones (IZs) over time\n\nThe ratio of listings in intermediate zones (IZs) over time\n\nThe ratio of li

('SELECT intermediateZone, month\nFROM listingIZMonthly\nWHERE year = 2020\nORDER BY listingRatio DESC\nLIMIT 1;',
                             intermediateZone     month
 0  Old Town, Princes Street and Leith Street  December,
 None)

We can execute the query in DuckDB to see if the result is correct. 

In [30]:
# con.sql("""
# SELECT listingRatio,intermediateZone, year, month
# FROM listingIZMonthly
# where year=2021
# ORDER BY listingRatio 
# ;
# """)
# LIMIT 1;

con.sql("""
SELECT intermediateZone, month FROM listingIZMonthly WHERE year = 2020 ORDER BY listingRatio DESC LIMIT 1;
""")

┌───────────────────────────────────────────┬──────────┐
│             intermediateZone              │  month   │
│                  varchar                  │ varchar  │
├───────────────────────────────────────────┼──────────┤
│ Old Town, Princes Street and Leith Street │ December │
└───────────────────────────────────────────┴──────────┘

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()

Your app is running at:
http://localhost:8084
 * Serving Flask app 'vanna.flask'
 * Debug mode: on
None
None


# Some final reflections
We demonstrated how databases can be queried by plain language questions through Text-to-Query technique.  </b>
This technique is still developing along with LLMs.  </b>
Before considering using these models, you also need to know the challenges and limitations. </b>

* Hallucinations </br>
A model generates output that is not based on the provided context or facts. For text-to-SQL tasks, this means the system creates SQL queries referencing non-existent columns, tables, or filters.</br>
The latest techniques to fix this include clarifying schema, Retrieval-Augmented Generation (RAG), Reasoning + Acting (ReAct) and Chain of Thought (CoT), etc.</b>

* Preformance </br>
Different SQL queries can produce the same result. This leads to challenges in calculating the efficiency of AI-generated SQL. </br>
Query large databases easily increase unexpected costs to use LLMs. 
Hard to implement models in multi-disciplinary teams, e.g directly use spatial joins in the example. 

* Security </br>
Generating SQL from NLP often requires access to the local data, e.g. 'allow_llm_to_see_data' in Vanna. </br>
Using public LLM is not an option. Running a local LLM also comprises security. LLM can be manipulated to reveal sensitive user information. 

Now, it is your turn for some exercise. </b>
Please 
* use the explore different training data based on **listing per KM area** for each intermediate zone per month year;
* try spatial SQLs if you have time;
* Anyone wants to try vectorDB? 

## Reference 
https://medium.com/wrenai/reducing-hallucinations-in-text-to-sql-building-trust-and-accuracy-in-data-access-176ac636e208 </br>
    
https://medium.com/wrenai/reducing-hallucinations-in-text-to-sql-building-trust-and-accuracy-in-data-access-176ac636e208 </b>

https://www.k2view.com/blog/llm-text-to-sql/#Reducing-the-risks-of-LLM-text-to-SQL-models